### Notebook que contiene los pasos para entrenar a un modelo de Mask R-CNN haciendo uso de un entrenamiento por capas y de la base de datos de sombras conocida como SBU-SHADOWS

In [ ]:
#Install needed packages (see requirements.txt in github repository)
!pip install numpy
!pip install "scipy==1.4.1"
!pip install Pillow
!pip install cython
!pip install matplotlib
!pip install scikit-image
!pip uninstall keras-nightly
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.15.0
!pip install keras==2.0.8
!pip install opencv-python
!pip uninstall h5py
!pip install h5py==2.10.0
!pip install imgaug
#Download model
!git clone https://github.com/nasca37/Mask_RCNN.git



In [ ]:
%cd Mask_RCNN/

#Download pre-trained weights 
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
%cd Mask_RCNN/
DATASET = 0

!wget https://www.dropbox.com/s/660s0vgyk83aek9/Img.zip
#!wget https://www.dropbox.com/s/w1henmfcu47a3cj/Mask.zip
#!wget https://www.dropbox.com/s/zz5u3kh3q4xremn/MaskTrain.zip
!wget https://www.dropbox.com/s/52ux32n0yu5xaq0/ImgTrain.zip

!wget https://www.dropbox.com/s/yes1a53053zi6pv/Maskjson.zip
!wget https://www.dropbox.com/s/brkhhd31cytt7z3/MaskTrainjson.zip

!unzip Img.zip
#!unzip Mask.zip
!unzip Maskjson.zip 
!unzip ImgTrain.zip
#!unzip MaskTrain.zip
!unzip MaskTrainjson.zip  


In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from matplotlib import pyplot
from csv import reader
import json
import numpy as np
import ntpath
import os
import sys
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.model import MaskRCNN

from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

DATASET_DIR_TEST=os.path.join(ROOT_DIR, "Img")
#DATASET_MASK_TEST=os.path.join(ROOT_DIR, "Mask")
DATASET_MASK_TEST=os.path.join(ROOT_DIR, "Maskjson")
DATASET_DIR_TRAIN=os.path.join(ROOT_DIR, "ImgTrain")
#DATASET_MASK_TRAIN=os.path.join(ROOT_DIR, "MaskTrain")
DATASET_MASK_TRAIN=os.path.join(ROOT_DIR, "MaskTrainjson")

Using TensorFlow backend.


In [ ]:
############################################################
#  Configurations
############################################################


class ShadowsConfig(Config):
    """Configuration for training on the   dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "Shadows"
    
    # A GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
#    IMAGES_PER_GPU = 2
    IMAGES_PER_GPU = 2
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + shadow
    
    # Number of training steps per epoch
    #STEPS_PER_EPOCH = 200
    STEPS_PER_EPOCH = 100
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    MINI_MASK_SHAPE = (128, 128)
    MASK_SHAPE = [56, 56]
    LOSS_WEIGHTS = {
      "rpn_class_loss":1.2,
      "rpn_bbox_loss": 0.7,
      "mrcnn_class_loss": 1.,
      "mrcnn_bbox_loss": 0.7,
      "mrcnn_mask_loss": 1.2
  }

In [ ]:
###############
# Define model
###############

# define the model
config = ShadowsConfig()
model = MaskRCNN(mode='training', model_dir=DEFAULT_LOGS_DIR, config=config)
# load weights (mscoco) and exclude the output layers
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
############################################################
#  Dataset
############################################################

import json
import numpy as np
import skimage
import re
class ShadowDataset(Dataset):

    def load_shadow(self, isTrain = True):
        """Load a subset of the nuclei dataset.

        dataset_dir: Root directory of the dataset
        subset: Subset to load. Either the name of the sub-directory,
                such as stage1_train, stage1_test, ...etc. or, one of:
                * train: stage1_train excluding validation images
                * val: validation images from VAL_IMAGE_IDS
        """
        # Add classes. We have one class.
        # Naming the dataset nucleus, and the class nucleus
        self.add_class("shadow", 1, "shadow")
        if isTrain:
          dataset_dir = DATASET_DIR_TRAIN
        else:
          dataset_dir = DATASET_DIR_TEST

        # Get image ids from directory names
        image_ids = next(os.walk(dataset_dir))[2]
        p = 1

        n=len(image_ids)
        all_indices=np.arange(n)
        ntrain=int(p*n)
        np.random.seed(100) #run this line before np.random.choice to choose always the same sequence of values
        fimage_ids=np.random.choice(n, ntrain, replace=False)
        
        # Add images
        for image_id in fimage_ids:
            self.add_image(
                "shadow",
                image_id=image_ids[image_id],
                path=os.path.join(dataset_dir, image_ids[image_id]))

        

    def load_mask(self, image_id,isTrain = True):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        id = re.findall(r'\d+', info['id'])[0]
        # Get mask directory from image path
        if isTrain == False:
          #mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "Mask/mask ({}).json".format(id))
          mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "Maskjson/mask ({}).json".format(id))
        else:
          #mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "MaskTrain/mask ({}).json".format(id))
          mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "MaskTrainjson/mask ({}).json".format(id))
          
        # Read mask info from .json file
        datashadows = json.load(open(mask_dir))
        # create one array for all masks, each on a different channel
        mask = np.zeros([datashadows["height"], datashadows["width"], len(datashadows["regions"])],
                        dtype=np.uint8)
             
        for i, p in enumerate(datashadows["regions"]):
            mask[p['y'], p['x'], i] = 1
        
        
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shadow":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
###############
# Check dataset
###############

# train set
train_set = ShadowDataset()
train_set.load_shadow( isTrain=True)
train_set.prepare()

#print info
# enumerate all images in the dataset
print(train_set.image_ids)
for image_id in train_set.image_ids:
	# load image info
	info = train_set.image_info[image_id]
	# display on the console
#	print(info)

[   0    1    2 ... 1631 1632 1633]


In [ ]:
#display masks
# load an image
image_id = 45
image = train_set.load_image(image_id)
print(image.shape)
# load image mask
mask, class_ids = train_set.load_mask(image_id,True)
print(mask.shape)
# plot image
pyplot.imshow(image)
# plot mask
pyplot.imshow(mask[:, :, 0], cmap='gray', alpha=0.5)
pyplot.show()

In [ ]:
# plot first few images
for i in range(9):
	# define subplot
	pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	image = train_set.load_image(i)
	pyplot.imshow(image)
	# plot all masks
	mask, _ = train_set.load_mask(i)
	for j in range(mask.shape[2]):
		pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)

# show the figure
pyplot.show()

In [ ]:
#display bounding boxes
# define image id
image_id = 1
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

In [ ]:
###################################################
#Draw ground truth masks on images from datasets 
###################################################
# from https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb

from mrcnn import visualize

# Training dataset.
dataset_train = ShadowDataset()
dataset_train.load_shadow( isTrain=True)
dataset_train.prepare()

# Validation dataset
dataset_val = ShadowDataset()
dataset_val.load_shadow( isTrain=False)
dataset_val.prepare()

image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# Test on a random image
from mrcnn.model import log

class InferenceConfig(ShadowsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

image_id = np.random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

#visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
 #                           dataset_train.class_names, figsize=(8, 8))

### Entrenamiento por capas


In [ ]:
###############
# Train
###############
from imgaug import augmenters as iaa
# Training dataset.
dataset_train = ShadowDataset()
dataset_train.load_shadow(isTrain=True)
dataset_train.prepare()

# Validation dataset
dataset_val = ShadowDataset()
dataset_val.load_shadow( isTrain=False)
dataset_val.prepare()


# train weights (output layers or 'heads')
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs=60, layers='heads')

In [ ]:
###############
# Define model
###############

# define the model
config2 = ShadowsConfig()
logs = os.path.join(DEFAULT_LOGS_DIR,"2")
model2 = MaskRCNN(mode='training', model_dir=logs, config=config2)
# load weights (mscoco) and exclude the output layers
LEARNED_WEIGHTS_PATH = os.path.join(DEFAULT_LOGS_DIR, "shadows20210315T1237/mask_rcnn_shadows_0059.h5")
model2.load_weights(LEARNED_WEIGHTS_PATH, by_name=True)

model2.train(dataset_train, dataset_val, learning_rate=config2.LEARNING_RATE/2, epochs=90, layers='3+')

In [ ]:
###############
# Define model
###############

# define the model
config3 = ShadowsConfig()
logs = os.path.join(DEFAULT_LOGS_DIR,"3")
model3 = MaskRCNN(mode='training', model_dir=logs, config=config2)
# load weights (mscoco) and exclude the output layers
LEARNED_WEIGHTS_PATH = os.path.join(DEFAULT_LOGS_DIR, "2/shadows20210127T1027/mask_rcnn_shadows_0089.h5")
model3.load_weights(LEARNED_WEIGHTS_PATH, by_name=True)

model3.train(dataset_train, dataset_val, learning_rate=config2.LEARNING_RATE*4, epochs=120, layers='all')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_save_name = 'classifierMaskRcnn(soloHead60).pt'
path = F"/content/drive/My Drive/{model_save_name}" 
model.keras_model.save_weights(path)

In [ ]:
def id_image(name):
  finalpath = "/content/Mask_RCNN/ImgTrain/" + name
  for id in train_set.image_ids:
    path = train_set.image_reference(id)
    if path == finalpath:
      return id

In [ ]:
def id_image_test(name):
  finalpath = "/content/Mask_RCNN/Img/" + name
  for id in test_set.image_ids:
    path = test_set.image_reference(id)
    if path == finalpath:
      return id

### Validación de los resultados


In [ ]:
###################################
#Evaluation of the trained model
###################################

# evaluate the mask rcnn model on the facesFDDB dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
import math

# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "shadow"
    # number of classes (background + eye + mouth)
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    NUM_CLASSES = 1 + 1
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		if not math.isnan(AP):
		    APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# load the train dataset
train_set = ShadowDataset()
train_set.load_shadow(isTrain=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = ShadowDataset()
test_set.load_shadow(isTrain=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir=DEFAULT_LOGS_DIR, config=cfg)
# load model weights
LEARNED_WEIGHTS_PATH = os.path.join(logs, "shadows20210626T0839/mask_rcnn_shadows_0119.h5")
model.load_weights(LEARNED_WEIGHTS_PATH, by_name=True)

# evaluate model on training dataset
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)


# evaluate model on test dataset
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

Train: 4085
Test: 637
Re-starting from epoch 59


In [ ]:
# evaluate model on test dataset
#test_mAP = evaluate_model(test_set, model, cfg)
#print("Test mAP: %.3f" % test_mAP)

In [ ]:
#########################################
#Draw results on an images from dataset
#########################################

from matplotlib.patches import Rectangle

# plot a number of photos with ground truth and predictions
def plot_actual_vs_predicted(dataset, model, cfg, n_images=5):
    # load image and mask
    for i in range(n_images):
        # load the image and mask
        image = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        # convert pixel values (e.g. center)
        scaled_image = mold_image(image, cfg)
        # convert image into one sample
        sample = expand_dims(scaled_image, 0)
        # make prediction
        yhat = model.detect(sample, verbose=0)[0]
        # define subplot
        pyplot.subplot(n_images, 2, i*2+1)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Actual')
        # plot masks
        for j in range(mask.shape[2]):
            pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.3)
        # get the context for drawing boxes
        pyplot.subplot(n_images, 2, i*2+2)
        # plot raw pixel data
        pyplot.imshow(image)
        pyplot.title('Predicted')
        ax = pyplot.gca()
        # plot each box
        for k in range(len(yhat['rois'])):
            box = yhat['rois'][k]
            #for box in yhat['rois']:
            # get coordinates
            y1, x1, y2, x2 = box
            # calculate width and height of the box
            width, height = x2 - x1, y2 - y1
            # create the shape
            colorshape='black'
            if yhat['class_ids'][k] == 1:
                colorshape = 'red'
            if yhat['class_ids'][k] == 2:
                colorshape = 'blue'	    
            rect = Rectangle((x1, y1), width, height, fill=False, color=colorshape)
            # draw the box
            ax.add_patch(rect)
    # show the figure
    pyplot.show()


# plot predictions for train dataset
plot_actual_vs_predicted(train_set, model, cfg)

# plot predictions for test dataset
plot_actual_vs_predicted(test_set, model, cfg)

### Comprobar rendimiento con 150 imagenes del conjunto de la base de datos

In [ ]:
###############################################################
#Draw results on an image from dataset (display mask and type)
###############################################################
imagelist = []

for i in range(150):
  image_id = i
  original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, cfg, 
                            image_id, use_mini_mask=False)

  # make prediction
  yhat = model.detect([original_image], verbose=1)[0]
  imagelist.append(yhat)
  visualize.display_instances(original_image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            dataset_val.class_names, yhat['scores'])

In [ ]:
for i in range(1):
  image_id = id_image("")
  original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, cfg, 
                            image_id, use_mini_mask=False)

  # make prediction
  yhat = model.detect([original_image], verbose=1)[0]

  visualize.display_instances(original_image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            dataset_train.class_names, yhat['scores'])

In [ ]:
###############################################################
#Draw results on an image from dataset (display mask and type)
###############################################################
for i in range(20):
  image_id = i
  original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, cfg, 
                            image_id, use_mini_mask=False)

  # make prediction
  yhat = model.detect([original_image], verbose=1)[0]

  visualize.display_instances(original_image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            dataset_train.class_names, yhat['scores'])

In [ ]:
###############################################################
#Draw results on an image from dataset (display mask and type)
###############################################################
for i in range(20):
  dir = "img ("+str(i+613)+").jpg"
  print(dir)
  image_id = id_image_test(dir)
  print(image_id)
  original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(test_set, cfg, 
                            image_id, use_mini_mask=False)

  # make prediction
  yhat = model.detect([original_image], verbose=1)[0]

  visualize.display_instances(original_image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            test_set.class_names, yhat['scores'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_save_name = 'classifierMaskRcnnNewL.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
model.keras_model.save_weights(path)

### Probar imagenes propias con los pesos entrenados


In [ ]:
#########################################
#Draw results on an image (display mask)
#########################################

import skimage.color
import skimage.io
import skimage.transform

#adapted from class Dataset load_image (in utils.py)
def load_image_prediction(img_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(img_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    
    return image
        
!wget https://e00-expansion.uecdn.es/assets/multimedia/imagenes/2017/02/05/14862885824518.jpg

img_path='./14862885824518.jpg'

image = load_image_prediction(img_path)  
# convert pixel values (e.g. center)
scaled_image = mold_image(image, cfg)
# convert image into one sample
sample = expand_dims(scaled_image, 0)
# make prediction
yhat = model.detect(sample, verbose=0)[0]

#display masks and scores on image
visualize.display_instances(image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            dataset_val.class_names, yhat['scores'])